# **실험개요**  
1) Variational Auto Encoder 구현  
2) 정상 데이터만을 이용하여 학습  
3) 정상 데이터와 비정상 데이터에 대한 재생성 데이터간 유사도 측정  
4) 유사도를 기반으로 비정상/정상 구분  

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "autoencoders"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

In [ ]:
class Sampling(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var)) * K.exp(log_var / 2) + mean

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)
# 사용할 라이브러리 선언
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

#for data preprocessing
from sklearn.decomposition import PCA

#for modeling
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split #training and testing data split
from keras.layers import Input, Dense
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.preprocessing import StandardScaler
#filter warnings
import warnings
warnings.filterwarnings("ignore")

RANDOM_SEED = 42
#data 로드
data=pd.read_csv("/content/drive/MyDrive/total_data/정상 심전도 신호 데이터.csv")

#data 분할
X_train,X_test=train_test_split(data,test_size=0.2,random_state=RANDOM_SEED) #train_test_split을 이용하여 전체 데이터에서 train용과 test용을 분리
X_train = X_train.astype(float) / 255
X_train=X_train.dropna(axis=1) # 데이터에서 NaN이 존재한다면 해당 열을 제거
X_test = X_test.astype(float) / 255
X_test=X_test.dropna(axis=1) # 데이터에서 NaN이 존재한다면 해당 열을 제거
print(X_train.shape)
input_dim = X_train.shape[1] # input 차원을 위하여 입력한 csv파일의 첫번째 shape값 사용
print(input_dim)

encoding_dim = input_dim

input_layer = Input(shape=(input_dim, ))

tf.random.set_seed(42)
np.random.seed(42)

codings_size = 10

K = keras.backend

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)
# 사용할 라이브러리 선언
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

#for data preprocessing
from sklearn.decomposition import PCA

#for modeling
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split #training and testing data split
from keras.layers import Input, Dense
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.preprocessing import StandardScaler
#filter warnings
import warnings
warnings.filterwarnings("ignore")

RANDOM_SEED = 42
#data 로드
#data_paf = pd.read_csv("/content/drive/My Drive/pafcut/merge_paf.csv",header=None)
data=pd.read_csv("/content/drive/My Drive/merge_real.csv")

#leave one out cross validation
from sklearn.model_selection import LeaveOneOut



#data 분할
X_train,X_test=train_test_split(data,test_size=0.2,random_state=RANDOM_SEED) #train_test_split을 이용하여 전체 데이터에서 train용과 test용을 분리
X_train = X_train.astype(float) / 255
X_train=X_train.dropna(axis=1) # 데이터에서 NaN이 존재한다면 해당 열을 제거
X_test = X_test.astype(float) / 255
X_test=X_test.dropna(axis=1) # 데이터에서 NaN이 존재한다면 해당 열을 제거
print(X_train.shape)
input_dim = X_train.shape[1] # input 차원을 위하여 입력한 csv파일의 첫번째 shape값 사용
print(input_dim)


encoding_dim = input_dim

input_layer = Input(shape=(input_dim, ))

tf.random.set_seed(42)
np.random.seed(42)

codings_size = 10

K = keras.backend
z = keras.layers.Dense(encoding_dim, activation="selu")(input_layer)
z = keras.layers.Dense(encoding_dim/2, activation="selu")(z)
codings_mean = keras.layers.Dense(codings_size)(z)
codings_log_var = keras.layers.Dense(codings_size)(z)
codings = Sampling()([codings_mean, codings_log_var])
variational_encoder = keras.models.Model(
    inputs=[input_layer], outputs=[codings_mean, codings_log_var, codings])

decoder_inputs = keras.layers.Input(shape=[codings_size])
x = keras.layers.Dense(encoding_dim/2, activation="selu")(decoder_inputs)
x = keras.layers.Dense(encoding_dim, activation="selu")(x)
variational_decoder = keras.models.Model(inputs=[decoder_inputs], outputs=[x])

_, _, codings = variational_encoder(input_layer)
reconstructions = variational_decoder(codings)
variational_ae = keras.models.Model(inputs=[input_layer], outputs=[reconstructions])

latent_loss = -0.5 * K.sum(
    1 + codings_log_var - K.exp(codings_log_var) - K.square(codings_mean),
    axis=-1)
variational_ae.add_loss(K.mean(latent_loss) / 784.)
variational_ae.compile(loss="mse", optimizer="Nadam",  metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="./",
                               verbose=0,
                               save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)
history = variational_ae.fit(X_train, X_train, epochs=200, batch_size=256,shuffle=True,
                             validation_data=(X_test, X_test), #batch_size=256 to 512 수정 --> 오히려 성능이 떨어짐
                             verbose=1,
                    callbacks=[checkpointer, tensorboard]).history
# 모델 저장
model_json = variational_ae.to_json() 
with open("model_second.json", "w") as json_file: 
  json_file.write(model_json)
variational_ae.save_weights("model_second.h5") 
print("Saved model to disk")

In [ ]:
from tensorflow.compat.v2.keras.models import model_from_json


json_file = open("/content/drive/MyDrive/model/model_vae.json", "r") 
loaded_model_json = json_file.read() 
json_file.close()

loaded_model = model_from_json(loaded_model_json,custom_objects={'Sampling': Sampling})

loaded_model.load_weights("/content/drive/MyDrive/model/model_vae.h5") 
print("Loaded model from disk")

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right');

In [ ]:
predictions = loaded_model.predict(X_test)
plt.plot(X_test.to_numpy()[3,:])
plt.plot(predictions[3,:])

In [ ]:
mse = np.mean(np.power(X_test - predictions, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse})
error_df.describe()

In [ ]:
import sklearn
x_sk=sklearn.metrics.pairwise.cosine_similarity(X_test.to_numpy(), predictions, dense_output=True)
print(x_sk.mean())

In [ ]:
sh=X_test.shape[0]

In [ ]:
print(sh)

In [ ]:
cutoff = 0.69 #문턱값
y_pred_classes_df2 = np.zeros(shape=(sh,)) # 0으로 채워진 넘파이 배열 생성
#print(y_pred_classes.shape) #(37069,)로 잘 됨
for k in range(sh):
  x_sk=sklearn.metrics.pairwise.cosine_similarity(X_test.to_numpy()[k,:].reshape(1,121), predictions[k,:].reshape(1,121), dense_output=True)
  if x_sk.mean()>cutoff: #문턱값보다 높다면 1로 채운다
    y_pred_classes_df2[k]=1

In [ ]:
from sklearn.model_selection import KFold # K-Fold Cross Validation 
from sklearn.model_selection import cross_val_score # 점수 평가
from sklearn.model_selection import cross_val_predict # 예측
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix # confusion matrix

In [ ]:
y_pred_classes_ori_df2=np.ones(shape=(sh,))
sns.heatmap(confusion_matrix(y_pred_classes_ori_df2,y_pred_classes_df2), annot=True, fmt = '1.0f') #정상 heatmap 그리기

# 비교를 위한 데이터 로드
### 정상 데이터를 이용하여 학습하였으므로 여기서 로드하는 데이터는 비정상

In [ ]:
data2=pd.read_csv("/content/drive/MyDrive/total_data/비정상 심전도 데이터.csv")
data2=data2.dropna(axis=1)

In [ ]:
X_train2,X_test2=train_test_split(data2,test_size=0.2,random_state=RANDOM_SEED) #train_test_split을 이용하여 전체 데이터에서 train용과 test용을 분리
X_train2 = X_train2.astype(float) / 255
#print(X_train2) # 데이터에 마지막 열을 읽어오는데 NaN이 존재한다
X_train2=X_train2.dropna(axis=1) # 데이터에서 NaN이 존재한다면 해당 열을 제거
#print(X_train2)
tmp=X_train2.to_numpy()
plt.plot(tmp[0,:]) # ecg 데이터 하나만 그래프로 도식화 해보자
X_test2 = X_test2.astype(float) / 255
X_test2=X_test2.dropna(axis=1) # 데이터에서 NaN이 존재한다면 해당 열을 제거

In [ ]:
plt.plot(tmp[100,:]) # ecg 데이터 하나만 그래프로 도식화 해보자
predictions2 = loaded_model.predict(X_test2)
plt.plot(predictions2[100,:])

In [ ]:
mse2 = np.mean(np.power(X_test2 - predictions2, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse2})
error_df.describe()

In [ ]:
import sklearn
x_sk2=sklearn.metrics.pairwise.cosine_similarity(X_test2.to_numpy(), predictions2, dense_output=True)
print(x_sk2)
print(x_sk2.mean())

In [ ]:
cutoff2 = 0.69 #문턱값
y_pred_classes_df22 = np.ones(shape=(sh,)) # 0으로 채워진 넘파이 배열 생성
#print(y_pred_classes.shape) #(37069,)로 잘 됨
for k in range(sh):
  x_sk2=sklearn.metrics.pairwise.cosine_similarity(X_test2.to_numpy()[k,:].reshape(1,121), predictions2[k,:].reshape(1,121), dense_output=True)
  if x_sk2.mean()<cutoff2: #문턱값보다 높다면 1로 채운다
    y_pred_classes_df22[k]=0

In [ ]:
y_pred_classes_ori_df22=np.zeros(shape=(sh,))
sns.heatmap(confusion_matrix(y_pred_classes_ori_df22,y_pred_classes_df22), annot=True, fmt = '1.0f') #정상 heatmap 그리기